In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import Image
import scipy
import time

from pptx import Presentation
from pptx.util import Inches

# Make sure that caffe is on the python path:
caffe_root = '/home/axj232/caffe/' 
import sys
sys.path.insert(0, caffe_root + 'python')

import caffe

prs = Presentation()
blank_slide_layout = prs.slide_layouts[6]
left = top = Inches(1)
height = Inches(5.5)



# configure plotting
plt.rcParams['figure.figsize'] = (10, 10)
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.cmap'] = 'gray'

In [ ]:
def make_image(data,outputname):
    fig = plt.figure()
    #fig.set_size_inches(5, 5)
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    plt.set_cmap('jet')
    ax.imshow(data, aspect = 'normal')
    plt.savefig(outputname)

In [ ]:
#load our fully convolutional network 
net_full_conv = caffe.Net('deploy_full.prototxt', 
                          'full_convolutional_net.caffemodel', caffe.TEST)

In [ ]:
#load our mean file and get it into the right shape
transformer = caffe.io.Transformer({'data': net_full_conv.blobs['data'].data.shape})

a = caffe.io.caffe_pb2.BlobProto()
file = open('DB_train.binaryproto','rb')
data = file.read()
a.ParseFromString(data)
means = a.data
means = np.asarray(means)
means = means.reshape(3, 32, 32)


transformer.set_mean('data',means.mean(1).mean(1))
transformer.set_transpose('data', (2,0,1))
transformer.set_channel_swap('data', (2,1,0))
transformer.set_raw_scale('data', 255.0)

In [ ]:
#set the mode to use the GPU
caffe.set_device(0)
caffe.set_mode_gpu()

In [ ]:
fname='./images/34077_H-E_0_fal_4_0.png'
im_orig = caffe.io.load_image(fname)
slide = prs.slides.add_slide(blank_slide_layout)
pic = slide.shapes.add_picture(fname, left, top, height=height)

nrow_in=im_orig.shape[0] #we'll be doing padding later, 
ncol_in=im_orig.shape[1] #so lets make sure we know the original size

#IMPORTANT: note here that the shape is 2000 x 2000, which is the same size 
#as we specified in our deploy_full text!

In [ ]:
#patch_size = 32 #the patch size that trained the network
#hpatch_size = patch_size / 2 #this is the size of the edges around the image


#im_orig = np.lib.pad(im_orig, ((hpatch_size, hpatch_size+displace_factor), \
#                               (hpatch_size, hpatch_size+displace_factor), \
#                               (0, 0)),  'symmetric')

print im_orig.shape 

In [ ]:
out = net_full_conv.forward_all(data=np.asarray([transformer.preprocess('data', im_orig)]))

In [ ]:
layer="norm1"

for fi in xrange(0,32):
    print fi
    make_image(net_full_conv.blobs[layer].data[0, fi],"test.png")
    slide = prs.slides.add_slide(blank_slide_layout)
    pic = slide.shapes.add_picture("test.png", left, top, height=height)
    

In [ ]:
prs.save('test.pptx')

In [ ]:
net_full_conv.blobs